In [ ]:
import sys
import os
ROOT_DIR = os.path.abspath("modelpath")

sys.path.append(ROOT_DIR)


import tensorflow as tf
import numpy as np
import json
import skimage.io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from mrcnn import model as modellib
from mrcnn.config import Config
from skimage.draw import polygon
from skimage.measure import find_contours
from matplotlib.patches import Polygon

class InferenceConfig(Config):
    NAME = "coco_inference"
    NUM_CLASSES = 1 + 4  
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# Создание и загрузка модели
config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir="pathlogs")
model.load_weights('weights.h5', by_name=True)

# Загрузка данных (формат COCO)
with open('testannatations', 'r') as f:
    coco_data = json.load(f)

image_ids = [image['id'] for image in coco_data['images']]
category_id_to_name = {category['id']: category['name'] for category in coco_data['categories']}

results = []
all_true_positives = 0
all_false_positives = 0
all_false_negatives = 0

# Вспомогательные функции
def compute_iou_mask(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    iou = intersection / union
    return iou

def calculate_ap(recalls, precisions):
    recalls = np.concatenate(([0.], recalls, [1.]))
    precisions = np.concatenate(([0.], precisions, [0.]))
    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = np.maximum(precisions[i - 1], precisions[i])
    indices = np.where(recalls[1:] != recalls[:-1])[0] + 1
    ap = np.sum((recalls[indices] - recalls[indices - 1]) * precisions[indices])
    return ap

def calculate_map(results, coco_data, iou_thresholds=np.arange(0.5, 1.0, 0.05)):
    APs = []
    for iou_threshold in iou_thresholds:
        AP = []
        print(iou_threshold)
        for category_id in category_id_to_name.keys():
            category_results = [res for res in results if res['category_id'] == category_id]
            category_annotations = [ann for ann in coco_data['annotations'] if ann['category_id'] == category_id]

            if len(category_annotations) == 0:
                continue

            TP = np.zeros(len(category_results))
            FP = np.zeros(len(category_results))
            matched_annotations = []

            for i, result in enumerate(category_results):
                image_id = result['image_id']
                pred_mask = result['segmentation']
                score = result['score']

                true_annotations = [ann for ann in category_annotations if ann['image_id'] == image_id]
                iou_max = 0
                match = None

                for ann in true_annotations:
                    true_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1]), dtype=np.uint8)
                    poly = np.array(ann['segmentation']).reshape(-1, 2)
                    rr, cc = polygon(poly[:, 1], poly[:, 0], true_mask.shape)
                    true_mask[rr, cc] = 1

                    iou = compute_iou_mask(pred_mask, true_mask)
                    if iou > iou_max and ann not in matched_annotations:
                        iou_max = iou
                        match = ann

                if iou_max >= iou_threshold:
                    TP[i] = 1
                    matched_annotations.append(match)
                else:
                    FP[i] = 1

            cum_TP = np.cumsum(TP)
            cum_FP = np.cumsum(FP)
            precisions = cum_TP / (cum_TP + cum_FP + np.finfo(float).eps)
            recalls = cum_TP / len(category_annotations)
            AP.append(calculate_ap(recalls, precisions))
            print(AP)
        if AP:
            APs.append(np.mean(AP))

    mAP = np.mean(APs) if APs else 0
    return mAP

# Делание предсказаний
for image_id in image_ids:
    image_data = next(item for item in coco_data['images'] if item['id'] == image_id)
    image_path = 'testpath' + image_data['file_name']
    
    image = skimage.io.imread(image_path)
    if image.ndim != 3:
        print(f"Skipping image {image_id} due to incorrect dimensions")
        continue

    if image.shape[-1] != 3:
        print(f"Skipping image {image_id} due to incorrect number of channels")
        continue

    result = model.detect([image])[0]
    true_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

    for i in range(len(result['class_ids'])):
        if result['class_ids'][i] == 3:
            result['class_ids'][i] = 4
        elif result['class_ids'][i] == 4:
            result['class_ids'][i] = 3
        elif result['class_ids'][i] == 2:
            result['class_ids'][i] = 2
        else:
            result['class_ids'][i] = 1       
    
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for ann in true_annotations:
        true_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        poly = np.array(ann['segmentation']).reshape(-1, 2)
        rr, cc = polygon(poly[:, 1], poly[:, 0], true_mask.shape)
        true_mask[rr, cc] = 1
        true_contours = find_contours(true_mask, 0.5)
        
        for verts in true_contours:
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", linewidth = 1.5, edgecolor='lime')
            ax.add_patch(p)
        plt.text(ann['bbox'][0], ann['bbox'][1] - 10, f"True: {category_id_to_name[ann['category_id']]}", color='lime', fontsize=12, weight='bold')

    for i in range(len(result['class_ids'])):
        bbox = result['rois'][i]
        score = result['scores'][i]
        category_id = result['class_ids'][i]
        
        pred_mask = result['masks'][:, :, i]
        iou_max = 0
        match = None
        
        for ann in true_annotations:
            true_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
            poly = np.array(ann['segmentation']).reshape(-1, 2)
            rr, cc = polygon(poly[:, 1], poly[:, 0], true_mask.shape)
            true_mask[rr, cc] = 1
            iou = compute_iou_mask(pred_mask, true_mask)
            if iou > iou_max:
                iou_max = iou
                match = ann
        
        if iou_max >= 0.7:
            true_positives += 1
            true_annotations.remove(match)
        else:
            false_positives += 1

        result_dict = {
            "image_id": image_id,
            "category_id": int(category_id),
            "bbox": [int(bbox[1]), int(bbox[0]), int(bbox[3] - bbox[1]), int(bbox[2] - bbox[0])],
            "segmentation": pred_mask,
            "score": float(score)
        }
        results.append(result_dict)
        
        pred_contours = find_contours(pred_mask, 0.5)
        for verts in pred_contours:
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", linewidth = 1.5, edgecolor='orangered')
            ax.add_patch(p)
        ax.add_patch(p)
        plt.text(bbox[1], bbox[0] + 10, f"Pred: {category_id_to_name[category_id]} {score:.2f}", color='orangered', fontsize=12, weight='bold')
        plt.text(bbox[1], bbox[0] - 30, f"IoU: {iou_max:.2f}", color='violet', fontsize=12, weight='bold')

    false_negatives += len(true_annotations)
    all_true_positives += true_positives
    all_false_positives += false_positives
    all_false_negatives += false_negatives

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f"Image ID: {image_id}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")
    print("\n")

    plt.xlabel(f" Image ID: {image_id} | TP: {true_positives} | FP: {false_positives} | FN: {false_negatives} \n Precision: {round(precision, 5)} | Recall: {round(recall, 5)} | F1 Score: {round(f1_score, 5)}", fontsize=14)
    plt.show()

# Вычисление общих метрик
overall_precision = all_true_positives / (all_true_positives + all_false_positives) if (all_true_positives + all_false_positives) > 0 else 0
overall_recall = all_true_positives / (all_true_positives + all_false_negatives) if (all_true_positives + all_false_negatives) > 0 else 0
overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1_score}")

mAP = calculate_map(results, coco_data)
print(f"mAP: {mAP}")